In [ ]:
import polars as pl

In [ ]:
import os

# กำหนด Environment Variable
os.environ["HF_HOME"] = "/data/models"
# ตัวอย่าง: os.environ["HF_HOME"] = "D:/HuggingFace_Data"

from datasets import load_dataset

test_ds = load_dataset("Porameht/processed-voice-th-169k")
print(f"Dataset cached at: {os.environ['HF_HOME']}/datasets/Porameht___processed-voice-th-169k/default/")

In [ ]:
Path.home() / '.kaggle' / 'kaggle.json'

In [ ]:
from datasets import load_dataset

ds = load_dataset("Porameht/processed-voice-th-169k") # โหลดทั้งหมด (train, validation, test)

In [4]:
# สมมติว่า ds คือ dataset ที่ยังไม่ได้แบ่ง
# ds_splits = ds.train_test_split(test_size=0.1, seed=42)
# train_ds = ds_splits["train"]
# test_ds = ds_splits["test"]

# ถ้าคุณโหลด Porameht/processed-voice-th-169k มา มันมี train, validation, test อยู่แล้ว
from datasets import load_dataset

train_ds = load_dataset("/data/models/datasets", split="train")
val_ds = load_dataset("/data/models/datasets", split="validation")
test_ds = load_dataset("/data/models/datasets", split="test")
print(test_ds)

Resolving data files:   0%|          | 0/24 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/24 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/24 [00:00<?, ?it/s]

Dataset({
    features: ['sentence', 'audio'],
    num_rows: 11983
})


In [4]:
import polars as pl

In [3]:
import pandas as pd

In [ ]:
# Convert to pandas first, then to polars
df = pl.from_pandas(test_ds.to_pandas())
df.write_parquet("/data/models/test_dataset.parquet")

In [13]:
ds = test_ds

In [14]:
import soundfile as sf
import os
from datasets import load_dataset # ต้อง import load_dataset ถ้ายังไม่มี ds ใน memory

output_dir = "/data/models/exported_audio"
os.makedirs(output_dir, exist_ok=True)

# ตรวจสอบจำนวนตัวอย่างทั้งหมดใน ds ก่อนที่จะวนลูป
num_samples = len(ds)
print(f"กำลังจะบันทึก {num_samples} ไฟล์เสียง...")

# ใช้ loop เพื่อบันทึกไฟล์ทั้งหมด
# ใช้ len(ds) แทน 11983 เพื่อให้โค้ดมีความยืดหยุ่นกับขนาด dataset
for i in range(num_samples):
    sample = ds[i]
    audio_array = sample["audio"]["array"]
    sampling_rate = sample["audio"]["sampling_rate"]
    # sentence = sample["sentence"] # ไม่ได้ใช้ในชื่อไฟล์ แต่เก็บไว้เผื่ออยากแสดงผล

    # ดึงชื่อไฟล์เดิมจากคอลัมน์ 'audio' -> 'path'
    original_filename_with_ext = sample["audio"]["path"]

    # สร้าง Path เต็มสำหรับไฟล์เอาต์พุต (ใช้ชื่อไฟล์เดิม)
    output_filename = os.path.join(output_dir, original_filename_with_ext)

    try:
        # พยายามบันทึกเป็น MP3
        # ต้องแน่ใจว่าระบบมี ffmpeg หรือ libsndfile ที่รองรับ MP3
        sf.write(output_filename, audio_array, sampling_rate, format='MP3')
        print(f"บันทึกสำเร็จ: {output_filename}")
    except Exception as e:
        print(f"ไม่สามารถบันทึก {output_filename} เป็น MP3 ได้: {e}")
        print(f"ลองบันทึกเป็น WAV แทน...")
        # ถ้า MP3 ล้มเหลว ลองบันทึกเป็น WAV แทน
        output_filename_wav = os.path.join(output_dir, os.path.splitext(original_filename_with_ext)[0] + ".wav")
        try:
            sf.write(output_filename_wav, audio_array, sampling_rate, format='WAV')
            print(f"บันทึกสำเร็จ (เป็น WAV): {output_filename_wav}")
        except Exception as e_wav:
            print(f"ไม่สามารถบันทึก {output_filename_wav} เป็น WAV ได้เช่นกัน: {e_wav}")

print("\nกระบวนการบันทึกไฟล์เสร็จสิ้น.")

กำลังจะบันทึก 11983 ไฟล์เสียง...
บันทึกสำเร็จ: /data/models/exported_audio/common_voice_th_25669098.mp3
บันทึกสำเร็จ: /data/models/exported_audio/common_voice_th_25669097.mp3
บันทึกสำเร็จ: /data/models/exported_audio/common_voice_th_25669285.mp3
บันทึกสำเร็จ: /data/models/exported_audio/common_voice_th_25669283.mp3
บันทึกสำเร็จ: /data/models/exported_audio/common_voice_th_25669355.mp3
บันทึกสำเร็จ: /data/models/exported_audio/common_voice_th_25669354.mp3
บันทึกสำเร็จ: /data/models/exported_audio/common_voice_th_25669289.mp3
บันทึกสำเร็จ: /data/models/exported_audio/common_voice_th_25669287.mp3
บันทึกสำเร็จ: /data/models/exported_audio/common_voice_th_25669356.mp3
บันทึกสำเร็จ: /data/models/exported_audio/common_voice_th_25669094.mp3
บันทึกสำเร็จ: /data/models/exported_audio/common_voice_th_25669096.mp3
บันทึกสำเร็จ: /data/models/exported_audio/common_voice_th_25669359.mp3
บันทึกสำเร็จ: /data/models/exported_audio/common_voice_th_25669001.mp3
บันทึกสำเร็จ: /data/models/exported_audio/co

In [9]:
test_ds = ds

In [12]:
import pandas as pd
from datasets import load_dataset # ต้อง import load_dataset ด้วยเพื่อสร้าง test_ds

df_output = pd.DataFrame({
    "audio": [item["path"] for item in test_ds["audio"]], # <--- แก้ไขตรงนี้!
    "sentence": test_ds["sentence"]
})

# 3. บันทึกเป็น CSV
output_csv_path = "/data/models/exported_sentences.csv" # ตรวจสอบ Path ให้แน่ใจว่าคุณมีสิทธิ์เขียน
df_output.to_csv(output_csv_path, index=False)

print(f"บันทึกข้อความเป็น CSV ที่: {output_csv_path}")
print("\nตัวอย่างข้อมูลใน CSV:")
print(df_output.head())

บันทึกข้อความเป็น CSV ที่: /data/models/exported_sentences.csv

ตัวอย่างข้อมูลใน CSV:
                          audio                                sentence
0  common_voice_th_25669098.mp3                       ฉันปลดเบรคได้พอดี
1  common_voice_th_25669097.mp3  ฉันสังเกตมานานแล้วว่าคุณกระหายที่จะรู้
2  common_voice_th_25669285.mp3            นี่เป็นการเดินทางครั้งที่ห้า
3  common_voice_th_25669283.mp3              ฉันติดหนี้เค้ามากเหลือเกิน
4  common_voice_th_25669355.mp3                              โทรฉุกเฉิน


In [7]:
old = pd.read_csv('/data/502507_pre/502507_pre/hcu-speech-recognition-challenge-2025/train.csv')
new = pd.read_csv('/data/models/exported_sentences.csv')

In [8]:
df_combined = pd.concat([old, new], ignore_index=True)

print("\nDataFrame ที่รวมกันแล้ว (ignore_index=True):")
print(df_combined)


DataFrame ที่รวมกันแล้ว (ignore_index=True):
                                    audio  \
0                     650019_AUG_0001.mp3   
1                     650019_AUG_0002.mp3   
2                     650019_AUG_0003.mp3   
3                     650019_AUG_0004.mp3   
4                     650019_AUG_0005.mp3   
...                                   ...   
13577  record_healthcare_7000019_1701.wav   
13578  record_healthcare_7000028_1333.wav   
13579   record_healthcare_7000038_847.wav   
13580  record_healthcare_7000044_1917.wav   
13581   record_healthcare_7000033_901.wav   

                                                sentence  
0      สาขาวิชากายภาพบำบัดเป็นวิชาชีพที่ต้องปฏิบัติโด...  
1      คณะกายภาพบำบัดของมหาวิทยาลัยหัวเฉียวเฉลิมพระเก...  
2      หลักสูตรที่เปิดสอนในคณะากายภาพบำบัด เป็นหลักสู...  
3              หลักสูตรของคณะนี้ใช้เวลาเรียนทั้งหมด 4 ปี  
4      นักศึกษาจะได้ฝึกปฏิบัติงานจริงในโรงพยาบาลทั่วป...  
...                                                  ...  


In [18]:
df_combined.to_csv('/data/Chuniji/texttospeech/mergerdatatot.csv')